In [5]:
import os
print(os.environ['LD_LIBRARY_PATH'])
os.environ['LD_LIBRARY_PATH'] = '/cvmfs/sft.cern.ch/lcg/releases/LCG_107a_ATLAS_2/tbb/2021.10.0/x86_64-el9-gcc13-opt/lib64:'+os.environ['LD_LIBRARY_PATH']
print(os.environ['LD_LIBRARY_PATH'])
import ROOT as r
import json

from pathlib import Path

/cvmfs/sft.cern.ch/lcg/releases/LCG_107a_ATLAS_2/tbb/2021.10.0/x86_64-el9-gcc13-opt/lib64/:/opt/root/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/lib64
/cvmfs/sft.cern.ch/lcg/releases/LCG_107a_ATLAS_2/tbb/2021.10.0/x86_64-el9-gcc13-opt/lib64:/cvmfs/sft.cern.ch/lcg/releases/LCG_107a_ATLAS_2/tbb/2021.10.0/x86_64-el9-gcc13-opt/lib64/:/opt/root/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/lib64


ImportError: libtbb.so.12: cannot open shared object file: No such file or directory

In [2]:
dataset_runnable = json.loads(Path("dataset_runnable/af_v3_marcus.json").read_text())
files_to_open=list(dataset_runnable['N2_220_N1_200_HH']['files'].keys())
metadata=dataset_runnable['N2_220_N1_200_HH']['metadata']

kFactor = metadata["kFactor"]
sumOfWeights = metadata["sum_of_weights"]
genFiltEff = metadata["genFiltEff"]
xs = metadata["crossSection"]
lum = metadata["luminosity"]

NameError: name 'json' is not defined

In [3]:
h_baseline_pt=r.TH1F("baseline_pt","baseline_pt",10000,0,10000)

h_istruth_it_ii_pt=r.TH1F("istruth_istight_isiso_pt","istight_isiso",10000,0,10000)
h_istruth_it_ni_pt=r.TH1F("istruth_istight_noiso_pt","istight_noiso",10000,0,10000)
h_istruth_nt_ii_pt=r.TH1F("istruth_notight_isiso_pt","notight_isiso",10000,0,10000)
h_istruth_nt_ni_pt=r.TH1F("istruth_notight_noiso_pt","notight_noiso",10000,0,10000)

h_notruth_it_ii_pt=r.TH1F("notruth_istight_isiso_pt","istight_isiso",10000,0,10000)
h_notruth_it_ni_pt=r.TH1F("notruth_istight_noiso_pt","istight_noiso",10000,0,10000)
h_notruth_nt_ii_pt=r.TH1F("notruth_notight_isiso_pt","notight_isiso",10000,0,10000)
h_notruth_nt_ni_pt=r.TH1F("notruth_notight_noiso_pt","notight_noiso",10000,0,10000)

In [7]:
eventcount=0
premindphicut=0
prejetscuts=0
postleadjetptcut=0
postmindphicut=0
postphotonpresel=0

postmetcut=0
postphtight=0
postlepveto=0

cutflowevents=0.
cutflowsumw=0.

filesopened=0
for file_to_open in files_to_open:
    filesopened+=1
    print(f"Processing file {file_to_open}, {filesopened} of {len(files_to_open)}")
    f=r.TFile(file_to_open,"RO")
    t=f.Get("analysis")
    totalevents=t.GetEntriesFast()
    for e in t:
    
        eventcount+=1
        if eventcount%50000 == 0:
            print(f"Processed {eventcount:6d}/{totalevents} events")
        
        #=================================================================
        # preselection
        #
        met=e.met_met_NOSYS
        met_phi=e.met_phi_NOSYS
        met_tlv=r.TLorentzVector()
        met_tlv.SetPtEtaPhiM(met,0,met_phi,0)
    
        # MET requirement, for now just look at low MET
        if met<200000.: 
            continue

        postmetcut+=1   

        # at least one loose photon with pT>10 GeV, no photons near jets
        photons={}
        photon_index=-1
        for p in range(len(e.ph_pt_NOSYS)):
            if not ord(e.ph_select_baseline_NOSYS[p])>0: 
                continue
    
            tlv=r.TLorentzVector()
            tlv.SetPtEtaPhiM(e.ph_pt_NOSYS[p],
                             e.ph_eta[p],
                             e.ph_phi[p],
                             0)
    
            if tlv.Pt()<10000.:
                continue
            
            if (abs(tlv.Eta())>2.37):
                continue

            if e.ph_author_NOSYS[p] != 4:
                continue
    
            # not clear we need this, should be done via overlap removal?  not applying for now.
            #mindrphjet=999
            #for j in jets:
            #    mindrphjet=min(mindrphjet,tlv.DeltaPhi(j))
            #if mindrphjet<0.4: continue
            
            if not ord(e.ph_select_or_dR04_NOSYS[p]) > 0:
                continue
                
            if photon_index<0:
                photon_index=p
                break
    
    
        if photon_index<0:
            continue

        photon_tlv = r.TLorentzVector()
        photon_tlv.SetPtEtaPhiM(e.ph_pt_NOSYS[photon_index],
                                e.ph_eta[photon_index],
                                e.ph_phi[photon_index],
                                0)
        
        postphtight+=1
        prejetscuts+=1
    
        # no b-tagged jets, at least one jet with pT>100 GeV, no jets near MET
        jets=[]
        mindphijetsmet=999.
        btags=0
        leadjetpt=0.
        ljpt=max(e.jet_pt_NOSYS)
        for j in range(len(e.jet_pt_NOSYS)):
            tlv=r.TLorentzVector()
            tlv.SetPtEtaPhiM(e.jet_pt_NOSYS[j],
                             e.jet_eta[j],
                             e.jet_phi[j],
                             0)
            if tlv.Pt()<20000:
                continue
            if abs(tlv.Eta())>2.8:
                continue
            if not (ord(e.jet_select_or_dR04_NOSYS[j]) > 0):
                continue
            
            jets.append(tlv)
            mindphijetsmet=min(mindphijetsmet,abs(tlv.DeltaPhi(met_tlv)))
            if ord(e.jet_GN2v01_FixedCutBEff_85_select[j])>0: 
                btags+=1
            
            leadjetpt=max(leadjetpt,e.jet_pt_NOSYS[j])
    
        if leadjetpt<150000.:
            continue
    
        postleadjetptcut+=1
        
        if btags>0: 
            continue
    
        premindphicut+=1
        
        if mindphijetsmet<0.4: 
            continue
    
        postmindphicut+=1            
        
        # veto events with leptons
        nelectrons = 0
        for l in range(len(e.el_pt_NOSYS)):
            tlv=r.TLorentzVector()
            tlv.SetPtEtaPhiM(e.el_pt_NOSYS[l],
                             e.el_eta[l],
                             e.el_phi[l],
                             0)
            if tlv.Pt()<4.5:
                continue
            if abs(tlv.Eta())>2.47:
                continue
            if not ord(e.el_select_baseline_NOSYS[l])>0: 
                continue
            if not ord(e.el_select_or_dR04_NOSYS[l])>0:
                continue
            nelectrons += 1

        nmuons     = 0
        for m in range(len(e.mu_pt_NOSYS)):
            tlv=r.TLorentzVector()
            tlv.SetPtEtaPhiM(e.mu_pt_NOSYS[m],
                             e.mu_eta[m],
                             e.mu_phi[m],
                             0)
            if tlv.Pt()<3.:
                continue
            if abs(tlv.Eta())>2.5:
                continue
            if not ord(e.mu_select_mediumID_NOSYS[m])>0: 
                continue
            if not ord(e.mu_select_or_dR04_NOSYS[m])>0:
                continue
            nmuons += 1   
            
        if (nelectrons+nmuons)>0: 
            continue

        postlepveto+=1

        mT_A = r.TMath.Sqrt(2*photon_tlv.Pt()*met*(1.-r.TMath.Cos(photon_tlv.DeltaPhi(met_tlv))))
        #print(mT_A)
        if mT_A > 50000.:
            continue

        ## overlap removal for vgamma samples
        #if not e.event_select_vgammaOverlap7: 
        #    continue
        
        h_baseline_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
    
        #=================================================================
    
        #=================================================================
        # now we have the index of the photon to keep.  Now figure out if this is a truth-matched photon or not.
        # will need to fix this whenever Giordon et al decide on a truth matching scheme.
        #print(f"{e.ph_truthType[photon_index]} {e.ph_truthOrigin[photon_index]} {e.ph_truthpdgId[photon_index]}")
        ph_truthmatch=((e.ph_truthType[photon_index]!=16) and (e.ph_truthType[photon_index]!=0))
    
        # the tight and isolated flags are easier:
        ph_tight=(ord(e.ph_select_tightID_NOSYS [photon_index])>0)
        ph_iso  =(ord(e.ph_select_tightIso_NOSYS[photon_index])>0)
        
        weight_norm_mc = xs*genFiltEff*kFactor*lum/sumOfWeights
        jetSF          = e.weight_fjvt_effSF_NOSYS * e.weight_ftag_effSF_GN2v01_Continuous_NOSYS * e.weight_jvt_effSF_NOSYS
        photonSF       = e.ph_id_effSF_tightID_NOSYS[photon_index] * e.ph_id_effSF_tightIso_NOSYS[photon_index]
        weight         = e.weight_mc_NOSYS * e.weight_pileup_NOSYS * weight_norm_mc * jetSF * photonSF
        
        if ph_tight and ph_iso:    
            cutflowevents += 1.
            cutflowsumw   += weight
            #print(e.eventNumber)
        
        if ph_truthmatch:
            if       ph_tight and     ph_iso: h_istruth_it_ii_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
            elif     ph_tight and not ph_iso: h_istruth_it_ni_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
            elif not ph_tight and     ph_iso: h_istruth_nt_ii_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
            elif not ph_tight and not ph_iso: h_istruth_nt_ni_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
        else:
            if       ph_tight and     ph_iso: h_notruth_it_ii_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
            elif     ph_tight and not ph_iso: h_notruth_it_ni_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
            elif not ph_tight and     ph_iso: h_notruth_nt_ii_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
            elif not ph_tight and not ph_iso: h_notruth_nt_ni_pt.Fill(e.ph_pt_NOSYS[photon_index]/1000.)
        #=================================================================

Processing file /data/kratsg/radiative-decays/v3/user.bhodkins.RadiativeAnalysis.545777.N2_220_N1_200_HH.mc20a.v3.0_ANALYSIS.root/user.bhodkins.44023013._000001.ANALYSIS.root, 1 of 3
5636
4452
5025
5571
4403
4770
1317
8530
102
143
790
7893
7097
7636
7845
7442
6072
7909
8939
8096
8660
9939
8739
4928
5531
5254
5701
4432
185
1776
1635
889
1547
7350
7851
6557
6949
7312
6473
7183
7818
7272
6738
6703
7175
830
1230
535
846
1528
1178
3910
2510
4108
4398
5329
7372
5372
9680
8082
8830
9649
5073
3699
3968
3333
3619
7664
9275
9471
8800
3278
2022
2967
3438
2118
2954
2101
2718
3965
8291
74
711
1371
77
1310
1613
1333
374
9660
8666
8986
9317
8434
9182
9493
7655
6693
6995
7388
401
1308
6185
6016
6487
6623
7237
1089
7650
6502
6435
6110
7019
6584
5662
5340
3199
3562
3478
3738
2762
2999
2644
8076
9272
2068
2793
3643
2243
1543
1602
9809
9335
773
3666
3956
2883
3707
3202
2372
2422
8715
8197
8630
896
1549
7511
7721
6541
7027
7635
9197
6944
7600
7559
3090
3049
8430
9373
9629
4252
5727
4456
4822
8313
8554
9745

Error in <THashList::Delete>: A list is accessing an object (0x8198040) already deleted (list name = THashList)
Error in <THashList::Delete>: A list is accessing an object (0x7fbcd0012b70) already deleted (list name = THashList)
Error in <THashList::Delete>: A list is accessing an object (0x7fbcd00128a0) already deleted (list name = THashList)


In [5]:
print(f"total events  {eventcount}")
print(f"post met      {postmetcut}")
print(f"post phtight  {postphtight}")
print(f"pre jets cuts {prejetscuts}") 

print(f"post leadjet  {postleadjetptcut}")
print(f"post btagveto {premindphicut}")  
print(f"post mindphi  {postmindphicut}")
print(f"post lepveto  {postlepveto}")
print(f"post photonpr {postphotonpresel}")
print(f"baseline:     {h_baseline_pt.GetSumOfWeights()}")
print(f"cutflow events:  {cutflowevents}")
print(f"cutflow sumw:    {cutflowsumw}")

print(f"A:  truth,  tight, !isolated: {h_istruth_it_ni_pt.GetSumOfWeights()}")
print(f"B:  truth, !tight, !isolated: {h_istruth_nt_ni_pt.GetSumOfWeights()}")
print(f"C:  truth,  tight,  isolated: {h_istruth_it_ii_pt.GetSumOfWeights()}")
print(f"D:  truth, !tight,  isolated: {h_istruth_nt_ii_pt.GetSumOfWeights()}")

print(f"A: !truth,  tight, !isolated: {h_notruth_it_ni_pt.GetSumOfWeights()}")
print(f"B: !truth, !tight, !isolated: {h_notruth_nt_ni_pt.GetSumOfWeights()}")
print(f"C: !truth,  tight,  isolated: {h_notruth_it_ii_pt.GetSumOfWeights()}")
print(f"D: !truth, !tight,  isolated: {h_notruth_nt_ii_pt.GetSumOfWeights()}")

total events  14382
post met      14382
post phtight  9865
pre jets cuts 9865
post leadjet  8134
post btagveto 7509
post mindphi  6654
post lepveto  5819
post photonpr 0
baseline:     1918.0
cutflow events:  1303.0
cutflow sumw:    232.9457529147704
A:  truth,  tight, !isolated: 323.0
B:  truth, !tight, !isolated: 82.0
C:  truth,  tight,  isolated: 1293.0
D:  truth, !tight,  isolated: 181.0
A: !truth,  tight, !isolated: 6.0
B: !truth, !tight, !isolated: 17.0
C: !truth,  tight,  isolated: 10.0
D: !truth, !tight,  isolated: 6.0


In [6]:
(233.6-232.95)/233.6

0.002782534246575367